In [2]:
from datenguide_python.query_execution import QueryExecutioner
import re
from functools import partial

In [5]:
class Field(object):
    internal_exec = QueryExecutioner()
    
    def __init__(self,name,return_Type,args=dict()):
        self.name = name
        self.subfields = []
        self.args = args
        self._returnType = return_Type
        
    def __str__(self):
        arg_str = str(self.args) if len(self.args) > 0 else ''
        return '\n'.join([self.name + arg_str] 
                         + [re.sub(r'(^|\n)',r'\1   ',str(sf))
                            for sf in self.subfields])
    
    def __getattr__(self,k):
        return partial(self.add_subfield,k)
    
    def __dir__(self):
        meta = self.internal_exec.get_type_info(self._returnType)
        return (super().__dir__() + list(meta.fields.keys()))
            
        
    @classmethod
    def toplevel(cls):
        return cls('query','Query')
    
    def info(self):
        return self.internal_exec._get_type_info(self._returnType)
        
    def add_subfield(self,field,args = {}):
        possible_fields = self.internal_exec.get_type_info(self._returnType).fields
        return_type = possible_fields[field].get_return_type()
        f = Field(field,return_type,args)
        self.subfields.append(f)
        return f
    

## .add_subfield style adding

In [6]:
query = Field.toplevel()
region = query.add_subfield('region',{'id':'01'})
n = region.add_subfield('name')
statistic = region.add_subfield('AENW01')
statistic2 = region.add_subfield('AENW06')
# Scalar fields don't need to be bound to variables as nothing can be done with them
y = statistic.add_subfield('year')
statistic2.add_subfield('year')
print(query)

query
   region{'id': '01'}
      name
      AENW01
         year
      AENW06
         year


## `__getattr__` style adding
This is also supported by the `__dir__` implementation of the class

In [7]:
query2 = Field.toplevel()
region2 = query2.region({'id':'01'})
n = region2.add_subfield('name') #namin conflict with attribute name of the field
statistic_2 = region2.AENW01()
statistic2_2 = region2.AENW06()
# Scalar fields don't need to be bound to variables as nothing can be done with them
y = statistic_2.year()
statistic2_2.year()
print(query2)

query
   region{'id': '01'}
      name
      AENW01
         year
      AENW06
         year


## Usefull executioner functionality
To build a complex query builder Field class using meta data the query executioner proveds the  `.get_type_info` method which returns a named tuple.

In [12]:
ex = QueryExecutioner()
stat_info = ex.get_type_info('AENW01')

In [14]:
stat_info.kind

'OBJECT'

In [15]:
stat_info.fields

{'id': {'name': 'id',
  'type': {'ofType': None,
   'kind': 'SCALAR',
   'name': 'String',
   'description': 'The `String` scalar type represents textual data, represented as UTF-8 character sequences. The String type is most often used by GraphQL to represent free-form human-readable text.'},
  'description': 'Interne eindeutige ID',
  'args': []},
 'year': {'name': 'year',
  'type': {'ofType': None,
   'kind': 'SCALAR',
   'name': 'Int',
   'description': 'The `Int` scalar type represents non-fractional signed whole numeric values. Int can represent values between -(2^31) and 2^31 - 1. '},
  'description': 'Jahr des Stichtages',
  'args': []},
 'value': {'name': 'value',
  'type': {'ofType': None,
   'kind': 'SCALAR',
   'name': 'Float',
   'description': 'The `Float` scalar type represents signed double-precision fractional values as specified by [IEEE 754](https://en.wikipedia.org/wiki/IEEE_floating_point). '},
  'description': 'Wert',
  'args': []},
 'source': {'name': 'source',
 

In [16]:
stat_info.enum_values

In [26]:
enum_info = ex.get_type_info('WHGGR1')

In [27]:
enum_info.kind,enum_info.fields

('ENUM', None)

In [28]:
enum_info.enum_values

{'WHGRME01': 'Wohnungen mit 1 Raum',
 'WHGRME02': 'Wohnungen mit 2 Räumen',
 'WHGRME03': 'Wohnungen mit 3 Räumen',
 'WHGRME04': 'Wohnungen mit 4 Räumen',
 'WHGRME05': 'Wohnungen mit 5 Räumen',
 'WHGRME06': 'Wohnungen mit 6 Räumen',
 'WHGRME07UM': 'Wohnungen mit 7 Räumen oder mehr',
 'GESAMT': 'Gesamt'}

## FieldMetaDict
In order to improve internal readbility and usability the individual subdicts of the named tuples `.fields` position, are FieldMetaDicts. This is a custom Subclass of Dict that addtionally provides two methods to obtain certain meta information. One is `.get_return_type()` and the second is `.get_arguments`. These are supposed to help with implementations while still keeping all the regular dicitonary functionality in place to cover edge cases and similar.

### Note on testing with inheritance
Due to subclassing `dict` and the is the following distinction in testing the class of a `FieldMetaDict` instance `x`.
```python
type(x) == dict  #returns false
isinstance(x,dict)  #returns still true
```

### Return types
`.get_return_type()` type modifiers of the return type like returning a list or elements for instance and always returns only the name of the underlying type of whatever is returned.

In [17]:
region_info = ex.get_type_info('Region')
query_info = ex.get_type_info('Query')

In [30]:
region_info.fields['WOHNY1'].get_return_type()

'WOHNY1'

In [39]:
(query_info.fields['region'].get_return_type()
 ,query_info.fields['allRegions'].get_return_type())

('Region', 'RegionsResult')

In [40]:
region_info.fields['id'].get_return_type()

'String'

### Argument Examples
`.get_arguments` returns a dict of all the arguments as well as some information about them. The information make take a different form depending on the argument being a LIST, NON_NULL or  having no special property. Currently the focus was on simply presenting this information to get a feling for it. Accessing it more systematically and providing more of an interface may come at a later point.

In [21]:
region_info.fields['WOHNY1'].get_arguments()

{'year': ('LIST', None, 'Int', 'SCALAR'),
 'statistics': ('LIST', None, 'WOHNY1Statistics', 'ENUM'),
 'WHGGR1': ('LIST', None, 'WHGGR1', 'ENUM'),
 'filter': ('INPUT_OBJECT', 'WOHNY1Filter', None, None)}

In [35]:
region_info.fields['AENW01'].get_arguments()

{'year': ('LIST', None, 'Int', 'SCALAR'),
 'statistics': ('LIST', None, 'AENW01Statistics', 'ENUM')}

In [24]:
query_info.fields['region'].get_arguments()

{'id': ('NON_NULL', None, 'String', 'SCALAR')}

In [25]:
query_info.fields['allRegions'].get_arguments()

{'page': ('SCALAR', 'Int', None, None),
 'itemsPerPage': ('SCALAR', 'Int', None, None)}